# Token Swap Using Marlowe Runtime\'s REST API

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The token-swap contract example is a simple Marlowe contract that lets parties trade ada for a token.

[A video works through this Jupyter notebook.](https://youtu.be/sSrVCRNoytU)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.

![Marlowe contract for swapping tokens](../../images/swap-playground.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Ada provider")
            (Role "Ada provider")
            (Token "" "")
            (MulValue
                (Constant 1000000)
                (ConstantParam "Amount of Ada")
            )
        )
        (When
            [Case
                (Deposit
                    (Role "Dollar provider")
                    (Role "Dollar provider")
                    (Token "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe" "Djed_testMicroUSD")
                    (ConstantParam "Amount of dollars")
                )
                (Pay
                    (Role "Ada provider")
                    (Party (Role "Dollar provider"))
                    (Token "" "")
                    (MulValue
                        (Constant 1000000)
                        (ConstantParam "Amount of Ada")
                    )
                    (Pay
                        (Role "Dollar provider")
                        (Party (Role "Ada provider"))
                        (Token "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe" "Djed_testMicroUSD")
                        (ConstantParam "Amount of dollars")
                        Close 
                    )
                )]
            (TimeParam "Timeout for dollar deposit")
            Close 
        )]
    (TimeParam "Timeout for Ada deposit")
    Close
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `MARLOWE_RT_WEBSERVER_HOST`: IP address of the Marlowe Runtime web server.
- `MARLOWE_RT_WEBSERVER_PORT`: Port number for the Marlowe Runtime web server.

It also assumes that the parties have addresses, signing keys, and funds.
- Ada Provider
    - [keys/lender.address](keys/lender.address): Cardano address for the ada provider
    - [keys/lender.skey](keys/lender.skey): location of signing key file for the ada provider
- Dollar Provider
    - [keys/borrower.address](keys/borrower.address): Cardano address for the dollar provider
    - [keys/borrower.skey](keys/borrower.skey): location of signing key file for the dollar provider

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [ ]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for Marlowe Runtime REST API.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST":"$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

Note the test network magic number:
- `preprod` = 1
- `preview` = 2

### Ada Provider address and funds

Check that an address and key has been created for the ada provider. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
ADA_PROVIDER_SKEY=keys/lender.skey
ADA_PROVIDER_ADDR=$(cat keys/lender.address)
echo "ADA_PROVIDER_ADDR = $ADA_PROVIDER_ADDR"

Check that the ada provider has at least three hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$ADA_PROVIDER_ADDR"

### Dollar Provider address and funds

Check that an address and key has been created for the dollar provider. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
USD_PROVIDER_SKEY=keys/borrower.skey
USD_PROVIDER_ADDR=$(cat keys/borrower.address)
echo "USD_PROVIDER_ADDR = $USD_PROVIDER_ADDR"

Check that the dollar provider has at least one hundred USD tokens.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$USD_PROVIDER_ADDR"

## Design the contract

The swap contract can be downloaded from the [Marlowe Playground](https://play.marlowe.iohk.io//) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Set the ada amount to 294 and the dollar amount to 100.000000.

In [ ]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
LOVELACE_AMOUNT=$((294 * ADA))
MICROUSD_AMOUNT=$((100 * 1000000))
echo "LOVELACE_AMOUNT = $LOVELACE_AMOUNT lovelace"
echo "MICROUSD_AMOUNT = $MICROUSD_AMOUNT µUSD"

On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [ ]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

Later in the example we will need some constants for converting times.

In [ ]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

Now design the contract.

1. Visit https://play.marlowe.iohk.io// in a web browser.
2. Select "Open an Example".
3. Select "Marlowe" or "Blockly" under "Swap".
4. Edit the contract so that the dollar token has the policy ID and name for the dollar token. In this example we use test djed, which has policy ID `9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe` and token name `Djed_testMicroUSD`.
5. Select "Send to Simulator".
6. Set the "Timeout for ada deposit" to one hour into the future.
7. Set the "Timeout for dollar deposit" to two hours into the future
8. Set "Amount of Ada" to 294.
9. Set "Amount of dollars" to 100,000,000, since the units of measure are millionths.
10. Select "Download as JSON", set the file name to "swap-contract.json", and store the file in this folder, namely [marlowe-starter-kit/](.).

![Setting parameters for the swap contract in Marlowe Playground](../../images/swap-simulation.png)

## Examine the contract

View the contract file as YAML.

In [ ]:
json2yaml swap-contract.json

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1: Ada provider Creates Swap Contract with Initial ADA

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [ ]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat swap-contract.json`
roles:
  Ada provider: "$ADA_PROVIDER_ADDR"
  Dollar provider: "$USD_PROVIDER_ADDR"
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

Next we post the request and view the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

The identifier for the contract is embedded in the response.

In [ ]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

The CBOR serialization (in text-envelope format) is also embedded in the response.

In [ ]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [ ]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

One can view the transaction on a Cardano explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

One can see that the ada and dollar providers have received their role tokens. Note that `4164612070726f7669646572 = "Ada provider`, `4275796572 = Buyer`, and `446f6c6c61722070726f7669646572 = "Dollar provider"` in hexadecimal notation. Also note that `446a65645f746573744d696372 = Djed_testMicroUSD`.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

In [ ]:
curl -sS "$CONTRACT_URL" | json2yaml

## Transaction 2: The Ada Provider Deposits 294 Ada into the Contract

The ada provider deposits their 294 ada into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The buyer is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [ ]:
marlowe-cli input deposit \
  --deposit-party 'Ada provider' \
  --deposit-account 'Ada provider' \
  --deposit-amount "$LOVELACE_AMOUNT" \
  --out-file input-2.json
json2yaml input-2.json

In [ ]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [ ]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

One can see that the ada provider has approximately 294 ada less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

The Marlowe contract still has the 2 ada from its creation and an additional 294 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_2" | json2yaml

## Transaction 3: The Dollar Provider deposits 100 Djed into the Contract

The dollar provider deposits their 100 djed into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The buyer is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [ ]:
marlowe-cli input deposit \
  --deposit-party 'Dollar provider' \
  --deposit-account 'Dollar provider' \
  --deposit-token 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.Djed_testMicroUSD \
  --deposit-amount "$MICROUSD_AMOUNT" \
  --out-file input-3.json
json2yaml input-3.json

In [ ]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
cat request-3.json

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $USD_PROVIDER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [ ]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$USD_PROVIDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

One can see that the dollar provider has exactly 100 djed less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

The Marlowe contract has closed, but the roll-payout address holds 100 djed for the benefit of the ada provider.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#2"

The roll-payout address also holds 294 ada for the benefit of the dollar provider.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#3"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_3" | json2yaml

## Transaction 4: The Ada Provider Withdraws the Dollars

The 100 djed is held at Marlowe's role-payout address for the benefit of the ada provider. The ada provider can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [ ]:
yaml2json << EOI > request-4.json
contractId: "$CONTRACT_ID"
role: "Ada provider"
EOI
cat request-4.json

Next we post the request and store the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [ ]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_4?tab=utxo"

The ada provider now has the 100 djed.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

## View the withdrawal

Marlowe Runtime\'s `HTTP` `GET` endpoint `/withdrawals/{transactionId}` can fetch a withdrawal from the blockchain and return information about it.

In [ ]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL"/withdrawals/"$TX_4" | json2yaml

## Transaction 5: The Dollar Provider Withdraws the Ada

The 294 ada is held at Marlowe's role-payout address for the benefit of the dollar provider. The dollar provider can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [ ]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: "Dollar provider"
EOI
cat request-5.json

Next we post the request and store the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $USD_PROVIDER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [ ]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$USD_PROVIDER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_5?tab=utxo"

The dollar provider now has about an additional 294 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

## View the withdrawal

Marlowe Runtime\'s `HTTP` `GET` endpoint `/withdrawals/{transactionId}` can fetch a withdrawal from the blockchain and return information about it.

In [ ]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL"/withdrawals/"$TX_5" | json2yaml